In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [7]:
foundFile = False
while not foundFile:
    path = input('Enter your filepath to the data (.csv only, enter "exit" to end): ')
    if path == 'exit':
        break
    elif path[-4:] != '.csv':
        print("That's not a csv file!")
        continue
    try:
        df = pd.read_csv(path)
        foundFile = True
    except FileNotFoundError:
        print('File not found. Enter your filepath again.')
print('============ Column Names ============')
print(df.columns)
print('============ DataFrame ============')
print(df)

============ Column Names ============
Index(['student_id', 'age', 'gender', 'course', 'study_hours',
       'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality',
       'study_method', 'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')
============ DataFrame ============
       student_id  age  gender   course  study_hours  class_attendance  \
0               1   17    male  diploma         2.78              92.9   
1               2   23   other      bca         3.37              64.8   
2               3   22    male     b.sc         7.88              76.8   
3               4   20   other  diploma         0.67              48.4   
4               5   20  female  diploma         0.89              71.6   
...           ...  ...     ...      ...          ...               ...   
19995       19997   18   other      bba         6.50              71.3   
19996       19998   18    male    b.com         3.71              41.6   
19997       19999   

In [ ]:

d = input('Enter the column(s) to drop (Enter "#NONE" if none): ')
li = d.split(' ')
print(li)
if li != ['#NONE']:
    for c in li:
        try:
            df.drop(columns=[c])
        except KeyError:
            print('Drop column failed, the column was', c)
cols = df.columns[df.dtypes.eq('object')]
for c in cols:
    df[c] = df[c].astype('category').cat.codes
print(df)

['exit']
Drop column failed, the column was exit
       student_id  age  gender  course  study_hours  class_attendance  \
0               1   17       1       6         2.78              92.9   
1               2   23       2       5         3.37              64.8   
2               3   22       1       1         7.88              76.8   
3               4   20       2       6         0.67              48.4   
4               5   20       0       6         0.89              71.6   
...           ...  ...     ...     ...          ...               ...   
19995       19997   18       2       4         6.50              71.3   
19996       19998   18       1       0         3.71              41.6   
19997       19999   19       2       6         7.88              68.2   
19998       20000   19       1       4         4.60              76.3   
19999       20001   20       1       1         7.50              47.9   

       internet_access  sleep_hours  sleep_quality  study_method  \
0     

In [ ]:
while True:
    response = input('Enter the response variable: ')
    try:
        x = df.drop(columns=[response])
        y = df[response]
        
        break
    except KeyError:
        continue